In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import pandas_datareader.data as web

import pymc3 as pm
import arviz
from scipy import stats

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import gridspec
benchmark = web.DataReader('SP500', data_source='fred', start=2010)
benchmark.columns = ['benchmark']

with pd.HDFStore('data/assets.h5') as store:
    stock = store['quandl/wiki/prices'].adj_close.unstack()['AMZN'].to_frame('stock')
data = stock.join(benchmark).pct_change().dropna().loc['2010':]

data.info()

WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


OSError: ``data`` does not exist

In [ ]:
mean_prior = data.stock.mean()
std_prior = data.stock.std()
std_low = std_prior / 1000
std_high = std_prior * 1000

with pm.Model() as sharpe_model:
    mean = pm.Normal('mean', mu=mean_prior, sd=std_prior)
    std = pm.Uniform('std', lower=std_low, upper=std_high)

    nu = pm.Exponential('nu_minus_two', 1 / 29, testval=4) + 2.
    returns = pm.StudentT('returns', nu=nu, mu=mean, sd=std, observed=data.stock)

    sharpe = returns.distribution.mean / returns.distribution.variance ** .5 * np.sqrt(252)
    pm.Deterministic('sharpe', sharpe)
sharpe_model.model

In [ ]:
pm.model_to_graphviz(model=sharpe_model)

In [ ]:
tune = 2000
draws = 200
with sharpe_model:
    trace = pm.sample(tune=tune, draws=draws, chains=4, cores=1)

In [ ]:
trace_df = pm.trace_to_dataframe(trace).assign(chain=lambda x: x.index // draws)
trace_df.info()

In [ ]:
# arviz.plot_trace(data=trace);

In [ ]:
# Continue Sampling
draws = 25000
with sharpe_model:
    trace = pm.sample(draws=draws, trace=trace, chains=4, cores=1)

In [ ]:
pm.trace_to_dataframe(trace).shape

In [ ]:
df = pm.trace_to_dataframe(trace).iloc[400:].reset_index(drop=True).assign(chain=lambda x: x.index // draws)
trace_df = pd.concat([trace_df.assign(run=1), df.assign(run=2)])
trace_df.info()

In [ ]:
trace_df.tail(10)

In [ ]:
trace_df_long = pd.melt(trace_df, id_vars=['run', 'chain'])

In [ ]:
g = sns.FacetGrid(trace_df_long, col='variable', row='run', hue='chain', sharex='col', sharey=False)
g = g.map(sns.distplot, 'value', hist=False, rug=False)

In [ ]:
# arviz.plot_trace(data=trace);

In [ ]:
# arviz.plot_posterior(data=trace)

In [ ]:
# arviz.plot_forest(data=trace);

In [ ]:
data.describe()

In [ ]:
# Sharpe Ratio Comparison as a Probabilistic Model
group = {1: data.stock, 2: data.benchmark}
combined = pd.concat([g for i, g in group.items()])

# priors
mean_prior = combined.mean()
std_prior = combined.std()
std_low = std_prior / 1000
std_high = std_prior * 1000
T = 251 ** .5
mean, std, returns = {}, {}, {}
with pm.Model() as best:
    nu = pm.Exponential('nu_minus_two', 1 / 29, testval=4) + 2.
    for i in [1, 2]:
        mean[i] = pm.Normal(f'mean_g{i}', mu=mean_prior, sd=std_prior, testval=group[i].mean())
        std[i] = pm.Uniform(f'std_g{i}', lower=std_low, upper=std_high, testval=group[i].std())
        returns[i] = pm.StudentT(f'returns_g{i}', nu=nu, mu=mean[i], sd=std[i], observed=group[i])
        pm.Deterministic(f'vol_g{i}', returns[i].distribution.sd * T)
        pm.Deterministic(f'sharpe_g{i}', returns[i].distribution.mean / returns[i].distribution.sd * T)

    mean_diff = pm.Deterministic('mean diff', mean[1] - mean[2])
    pm.Deterministic('std diff', std[1] - std[2])
    pm.Deterministic('effect size', mean_diff / (std[i] ** 2 + std[2] ** 2) ** .5 / 2)

pm.model_to_graphviz(model=best)

In [ ]:
with best:
    trace = pm.sample(draws=10000, tune=2500, progressbar=True, cores=1)

In [ ]:
pm.trace_to_dataframe(trace).info()

In [ ]:
# Evaluating the Trace

burn = 0
trace = trace[burn:]

fig = plt.figure(figsize=(14, 8), constrained_layout=True)
gs = gridspec.GridSpec(4, 2, wspace=0.1, hspace=0.4)
axs = [plt.subplot(gs[i, j]) for i in [0, 1, 2] for j in [0, 1]]
axs.append(plt.subplot(gs[3, :]))

def distplot_w_perc(trace, ax):
    sns.distplot(trace, ax=ax)
    ax.axvline(stats.scoreatpercentile(trace, 2.5), color='0.5', label='2.5 and 97.5 percentiles')
    ax.axvline(stats.scoreatpercentile(trace, 97.5), color='0.5')

for i in [1, 2]:
    label = f'Group {i}'
    sns.distplot(trace[f'mean_g{i}'], ax=axs[0], label=label)
    sns.distplot(trace[f'vol_g{i}'], ax=axs[2], label=label)
    sns.distplot(trace[f'sharpe_g{i}'], ax=axs[4], label=label)

distplot_w_perc(trace['mean diff'], axs[1])
distplot_w_perc(trace['vol_g1'] - trace['vol_g2'], axs[3])
distplot_w_perc(trace['sharpe_g1'] - trace['sharpe_g2'], axs[5])

sns.distplot(trace['effect size'], ax=axs[6])
for p in [2.5, 97.5]:
    axs[6].axvline(stats.scoreatpercentile(trace['effect size'], p), color='0.5')

for i in range(5):
    axs[i].legend(loc=0, frameon=True, framealpha=0.5)

axs[0].set(xlabel='Mean', ylabel='Belief', yticklabels=[])
axs[1].set(xlabel='Difference of means', yticklabels=[])
axs[2].set(xlabel='Annual volatility', ylabel='Belief', yticklabels=[])
axs[3].set(xlabel='Difference of volatility', yticklabels=[])
axs[4].set(xlabel='Sharpe', ylabel='Belief', yticklabels=[])
axs[5].set(xlabel='Difference of Sharpes', yticklabels=[])
axs[6].set(xlabel='Difference of means normalized by volatility', ylabel='Belief', yticklabels=[])

In [ ]:

def plot_traces(traces, burnin=2000):
    '''
    Plot traces with overlaid means and values
    '''
    summary = arviz.summary(traces[burnin:])['mean'].to_dict()
    ax = arviz.plot_trace(traces[burnin:], figsize=(15, len(traces.varnames)*1.5), lines=summary)

    for i, mn in enumerate(summary.values()):
        ax[i, 0].annotate(f'{mn:.2f}', xy=(mn, 0), xycoords='data', xytext=(5, 10), textcoords='offset points',
                          rotation=90, va='bottom', fontsize='large', color='#AA0022')

In [ ]:
plot_traces(trace, burnin=0)